In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [18]:
import os

import pandas as pd
# import pytorch_lightning as pl
import spacy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
from tqdm.notebook import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [10]:
# look at the data
train_filename = '/kaggle/input/nlp-getting-started/train.csv'
tr_df = pd.read_csv(train_filename, engine='python')
display(tr_df.head())
print(tr_df.shape)
print(tr_df.target.value_counts())

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


(7613, 5)
0    4342
1    3271
Name: target, dtype: int64


# Torchtext

In [34]:
class spacy_tokenizer:
    def __init__(self, lang):
        self.lang = lang
        self.nlp = spacy.blank(self.lang)
        
    def __call__(self, text):
        return [token.text for token in self.nlp(text)]

tokenizer = spacy_tokenizer('en')
TEXT = torchtext.data.Field(tokenize=tokenizer, lower=True)
LABEL = torchtext.data.LabelField(use_vocab=False)

fields = [('id', None), ('keyword', None), ('location', None), ('text', TEXT), ('target', LABEL)]
dataset = torchtext.data.TabularDataset(path=train_filename, format='csv', fields=fields, skip_header=True)

In [35]:
# split into train and validation dataset.
tr_data, val_data = dataset.split(split_ratio=0.8)
print(len(tr_data), len(val_data))

6090 1523


In [36]:
# Build vocab
TEXT.build_vocab(tr_data)
print('num vocab', len(TEXT.vocab))

max_size = 20000
TEXT.build_vocab(tr_data, max_size=max_size)
print('num vocab', len(TEXT.vocab))

num vocab 19417
num vocab 19417


In [47]:
# iterator
batch_size = 32
tr_iter = torchtext.data.BucketIterator(tr_data, batch_size=batch_size, shuffle=True, device=device)
val_iter = torchtext.data.BucketIterator(val_data, batch_size=batch_size, shuffle=False, device=device)  # do not use train=False

# The network

In [48]:
class Rnn(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, output_size, bidirectional=False, n_layers=1, dropout=0, rnn_type='rnn'):
        super().__init__()
        self.bidirectional = bidirectional
        self.n_layers = n_layers
        self.rnn_type = rnn_type
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.encoder = getattr(nn, rnn_type.upper())(embedding_dim, hidden_dim, bidirectional=bidirectional, num_layers=n_layers, dropout=dropout)
        
        in_features = hidden_dim
        if self.bidirectional:
            in_features *= 2
            
        self.fc = nn.Linear(in_features=in_features, out_features=output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        if self.rnn_type.upper() == 'LSTM':
            x, (h_, c_) = self.encoder(x)
        else:
            x, h_ = self.encoder(x)
        x = self.fc(h_)
        
        return x

# Training

In [80]:
input_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 100
output_size = 1
bidirectional=False
n_layers=1
dropout=0
rnn_type='rnn'
lr = 0.0001
num_epochs = 20

model = Rnn(input_size, embedding_dim, hidden_dim, output_size, bidirectional=bidirectional, n_layers=n_layers, dropout=dropout, rnn_type=rnn_type).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()

for epoch in tqdm(range(num_epochs)):
    model.train()
    tr_loss = 0
    for idx, batch in enumerate(tr_iter):
        optimizer.zero_grad()
        
        x, y = batch
        
        logits = model(x)
        loss = criterion(logits.squeeze().to(float), y.to(float))
        tr_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    tr_loss /= (idx + 1)
    
    with torch.no_grad():
        model.eval()
        num_correct = 0
        val_loss = 0
        for idx, batch in enumerate(val_iter):
            x, y = batch
            
            logits = model(x)
            loss = criterion(logits.squeeze().to(float), y.to(float))
            val_loss = loss.item()
            
            
            prediction = torch.sigmoid(logits)
            prediction = (prediction >= 0.5).to(int).squeeze()
            num_correct += sum(prediction == y)
            
        val_loss = loss / (idx + 1)
        accuracy = num_correct.item() / len(val_data) * 100
    
    print(f'End epoch {epoch + 1}: Train loss {tr_loss:.6f}, Validation loss {val_loss:.6f}, Accuracy {accuracy:.4f} %')

End epoch 1: Train loss 0.686492, Validation loss 0.014709, Accuracy 56.9928 %
End epoch 2: Train loss 0.685640, Validation loss 0.014543, Accuracy 57.3211 %
End epoch 3: Train loss 0.684400, Validation loss 0.014557, Accuracy 57.7150 %

